In [ ]:
!nvidia-smi

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm import tqdm
from utils.SkinImageData import SkinImage
from utils.ConvNeXT_V1 import SkinClassifier
from utils.utils_main import UnNormalize, TrainTransform, TestTransform
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

## I. Training

### 1. Define

In [ ]:
path_folder_dataset_train = 'Data/ISIC2020/train'
label_names=['benign_skin','malignant_skin']
label_encoder = [0,1]

In [ ]:
trainsize = 384
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

In [ ]:
n_class = 2
n_epochs = 20
batch_size = 10
epochs_save_model = 1
path_save_model = 'model'

n_workers = os.cpu_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = SkinClassifier('convnext_small_in22ft1k',n_class)
model.to(device)

weights = torch.FloatTensor([1, 10]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.0001)
print("num_workers =", n_workers, ", device =", device)

In [ ]:
train_transform = TrainTransform(trainsize=trainsize, mean=mean, std=std)
unorm = UnNormalize(mean=mean, std=std)

### 2. Load data train

In [ ]:
dataset_train = SkinImage(path_folder_dataset_train, label_names, label_encoder, transform=train_transform)
print('train size: ', len(dataset_train), '\npath: ', path_folder_dataset_train)

In [ ]:
trainloader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                          shuffle=True, num_workers=n_workers)

### 3. Training

In [ ]:
for epoch in range(n_epochs):
    print('Epoch: ', epoch + 1,'/', n_epochs,' is training...')
    model.train()
    true_labels = []
    predicted_labels = []
    for batch_id, (x, y) in enumerate(tqdm(trainloader), start=1):
        x = x.to(device)
        y = y.to(device).float()  # Convert y to float
        optimizer.zero_grad()
        y_pred = model(x).float()  # Convert y_pred to float
        loss = criterion(y_pred, y.long())
        loss.backward()
        optimizer.step()
        true_labels.extend(y.cpu().numpy())
        probs = F.softmax(y_pred, dim=-1)
        predictions = torch.argmax(probs, dim=-1)
        predicted_labels.extend(predictions.cpu().numpy())

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Calculate precision
    precision = precision_score(true_labels, predicted_labels)

    # Calculate recall
    recall = recall_score(true_labels, predicted_labels)

    # F1 score
    f1 = 2 * (precision * recall) / (precision + recall)


    print(f'Epoch: {epoch + 1} - acc: {accuracy:.4f} - precision: {precision:.4f} - recall: {recall:.4f} - f1: {f1:.4f}')
    # save model
    if (epoch + 1) % epochs_save_model == 0:
        torch.save(model.state_dict(), path_save_model + '/model_epoch_' + str(epoch + 1) + '.pth')

## II. evaluate 

### 1. Define

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:

def predict_datatset(model, dataset, batch_size):
    model.eval()
    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for batch_id, (x, y) in enumerate(tqdm(dataset), start=1):
            x = x.to(device)
            y = y.to(device).float()  # Convert y to float
            y_pred = model(x).float()  # Convert y_pred to float
            true_labels.extend(y.cpu().numpy())
            probs = F.softmax(y_pred, dim=-1)
            predictions = torch.argmax(probs, dim=-1)
            predicted_labels.extend(predictions.cpu().numpy())
    return true_labels, predicted_labels

In [ ]:
n_class = 2
batch_size_test = 100
path_folder_dataset_test = 'Data/ISIC2020/test'
path_model = 'model/ep1/model_epoch_10.pth'

n_workers = os.cpu_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

new_model = SkinClassifier('convnext_small_in22ft1k',n_class)
new_model.to(device)
new_model.load_state_dict(torch.load(path_model))

### 2. load data test

In [ ]:
test_transform = TestTransform(trainsize=trainsize, mean=mean, std=std)
dataset_test = SkinImage(path_folder_dataset_test, label_names, label_encoder, transform=test_transform)
print('test size: ', len(dataset_test), '\npath: ', path_folder_dataset_test)

In [ ]:

testloader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size_test,
                                          shuffle=False, num_workers=n_workers)

### 3. Results

In [ ]:
test_true_labels, test_predicted_labels = predict_datatset(new_model, testloader, batch_size_test)

In [ ]:
# Calculate accuracy, f1, precision, recall
test_accuracy = accuracy_score(test_true_labels, test_predicted_labels)
test_precision = precision_score(test_true_labels, test_predicted_labels, average='macro')
test_recall = recall_score(test_true_labels, test_predicted_labels, average='macro')
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)
print(f'Test - acc: {test_accuracy:.4f} - precision: {test_precision:.4f} - recall: {test_recall:.4f} - f1: {test_f1:.4f}')

In [ ]:
print(classification_report(test_true_labels, test_predicted_labels))
cm = confusion_matrix(test_true_labels, test_predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[ 'Not Melanoma','Melanoma'])
disp.plot()
plt.show()